In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import numpy as np
import datetime

In [26]:
#Write a function to extract the tabular information from the given URL under the heading By Market Capitalization, and save it to a data frame.
url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
r = requests.get(url).text
soup = BeautifulSoup(r, 'html.parser')
table = soup.find_all('tbody')
table = table[0]
rows = table.find_all('tr')

In [27]:
table_attribs = ['Name','MC_USD_Billion']

In [28]:
count = 0
df = pd.DataFrame(columns = table_attribs)
data_dict = dict()

for row in rows:
    if count < 11:
        col = row.find_all('td')
        if len(col) != 0:
            data_dict = {'Name':col[1].contents[2].contents[0], 'MC_USD_Billion':col[2].contents[0].replace('\n','')}
            df1 = pd.DataFrame(data_dict, index = [0])
            df = pd.concat([df, df1])
    else:
        break

df['MC_USD_Billion'] = df['MC_USD_Billion'].astype(float)

df

,Name,MC_USD_Billion
0,JPMorgan Chase,432.92
0,Bank of America,231.52
0,Industrial and Commercial Bank of China,194.56
0,Agricultural Bank of China,160.68
0,HDFC Bank,157.91
0,Wells Fargo,155.87
0,HSBC Holdings PLC,148.90
0,Morgan Stanley,140.83
0,China Construction Bank,139.82
0,Bank of China,136.81


In [40]:
for row in rows:
    if count < 11:
        col = row.find_all('td')
        if len(col) != 0:
            bank_name = col[1].find_all('a')[1]['title']
            print(bank_name)

JPMorgan Chase
Bank of America
Industrial and Commercial Bank of China
Agricultural Bank of China
HDFC Bank
Wells Fargo
HSBC
Morgan Stanley
China Construction Bank
Bank of China


In [41]:
for row in rows:
    if count < 11:
        col = row.find_all('td')
        if len(col) != 0:
            market_cap = float(col[2].contents[0][:-1])
            print(market_cap)

432.92
231.52
194.56
160.68
157.91
155.87
148.9
140.83
139.82
136.81


In [5]:
ex_df = pd.read_csv(r'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\DataEngCourse\Week_2_PracticeProject\Graded_Project\exchange_rate.csv')
ex_df

,Currency,Rate
0,EUR,0.93
1,GBP,0.80
2,INR,82.95


In [6]:
# USD -> GBP conversion 
df['MC_GBP_Billion'] = round(ex_df['Rate'][1] * df['MC_USD_Billion'],2)

# USD -> EUR conversion
df['MC_EUR_Billion'] = round(ex_df['Rate'][0] * df['MC_USD_Billion'],2)

# USD -> INR conversion
df['MC_INR_Billion'] = round(ex_df['Rate'][2] * df['MC_USD_Billion'],2)

df

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,346.34,402.62,35910.71
0,Bank of America,231.52,185.22,215.31,19204.58
0,Industrial and Commercial Bank of China,194.56,155.65,180.94,16138.75
0,Agricultural Bank of China,160.68,128.54,149.43,13328.41
0,HDFC Bank,157.91,126.33,146.86,13098.63
0,Wells Fargo,155.87,124.70,144.96,12929.42
0,HSBC Holdings PLC,148.90,119.12,138.48,12351.26
0,Morgan Stanley,140.83,112.66,130.97,11681.85
0,China Construction Bank,139.82,111.86,130.03,11598.07
0,Bank of China,136.81,109.45,127.23,11348.39


In [7]:
df.to_csv(r'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\DataEngCourse\Week_2_PracticeProject\Graded_Project\Largest_banks_data.csv')

In [8]:
def load_to_db(df, database, table_name):
    conn = sqlite3.connect(database)
    df.to_sql(table_name, conn, index = False, if_exists='replace')
    conn.close()
    

conn = sqlite3.connect('Banks.db')
df.to_sql('Largest_banks', conn, if_exists='replace')

10

In [9]:
# This function keeps a log of activities/tasks done
def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open("./code_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')

In [10]:
log_progress('This is a test!')

In [11]:
# Function to query the table
def run_query(query_statement, database):
    conn = sqlite3.connect(database)
    cursor = conn.cursor()
    cursor.execute(query_statement)
    rows = cursor.fetchall()
    conn.close()
    return rows

query_statement = 'SELECT Name, MC_GBP_Billion FROM Largest_banks'
run_query(query_statement, "Banks.db")

[('JPMorgan Chase', 346.34),
 ('Bank of America', 185.22),
 ('Industrial and Commercial Bank of China', 155.65),
 ('Agricultural Bank of China', 128.54),
 ('HDFC Bank', 126.33),
 ('Wells Fargo', 124.7),
 ('HSBC Holdings PLC', 119.12),
 ('Morgan Stanley', 112.66),
 ('China Construction Bank', 111.86),
 ('Bank of China', 109.45)]

In [12]:
test = run_query(query_statement, "Banks.db")
pd.DataFrame(test)

,0,1
0,JPMorgan Chase,346.34
1,Bank of America,185.22
2,Industrial and Commercial Bank of China,155.65
3,Agricultural Bank of China,128.54
4,HDFC Bank,126.33
5,Wells Fargo,124.70
6,HSBC Holdings PLC,119.12
7,Morgan Stanley,112.66
8,China Construction Bank,111.86
9,Bank of China,109.45


In [13]:
def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format)  
    with open("./code_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')

In [14]:
df

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,346.34,402.62,35910.71
0,Bank of America,231.52,185.22,215.31,19204.58
0,Industrial and Commercial Bank of China,194.56,155.65,180.94,16138.75
0,Agricultural Bank of China,160.68,128.54,149.43,13328.41
0,HDFC Bank,157.91,126.33,146.86,13098.63
0,Wells Fargo,155.87,124.70,144.96,12929.42
0,HSBC Holdings PLC,148.90,119.12,138.48,12351.26
0,Morgan Stanley,140.83,112.66,130.97,11681.85
0,China Construction Bank,139.82,111.86,130.03,11598.07
0,Bank of China,136.81,109.45,127.23,11348.39


In [15]:
value_at_index_4 = df.iloc[4][['Name', 'MC_EUR_Billion']]

print(value_at_index_4)

Name              HDFC Bank
MC_EUR_Billion       146.86
Name: 0, dtype: object
